# Inspecting Ecore file for SysML v2 Metamodel

This notebook steps through a couple of different methods for loading up the SysML v2 Ecore metamodel. One with a library called pyEcore and another just using the raw lxml library.

In [1]:
import json

In [2]:
from pyecore.resources import ResourceSet, URI
from pyecore.ecore import EClass, EAttribute, EString, EObject, EProxy, EEnum

In [3]:
# From the quick start for PyEcore to load a file into the metaclass model

rset = ResourceSet()
sysml_resource = rset.get_resource(URI('metamodel/SysML.ecore'))
mm_root = sysml_resource.contents[0]
rset.metamodel_registry[mm_root.nsURI] = mm_root

types_resource = rset.get_resource(URI('metamodel/types.ecore'))
mm_root2 = types_resource.contents[0]
rset.metamodel_registry[mm_root.nsURI] = mm_root2

In [4]:
mm_root

Determine what the fields for an EPackage in this framework are:

In [5]:
dir(mm_root)

['eAnnotations',
 'eClassifiers',
 'eSubpackages',
 'eSuperPackage',
 'eURIFragment',
 'getEAnnotation',
 'getEClassifier',
 'name',
 'nsPrefix',
 'nsURI']

In [6]:
all_classes = mm_root.eClassifiers

Determine what the fields for an EClass in this framework are:

In [7]:
dir(all_classes[0])

['abstract',
 'allInstances',
 'defaultValue',
 'eAllAttributes',
 'eAllGenericSuperTypes',
 'eAllGenericSuperTypesClassifiers',
 'eAllOperations',
 'eAllReferences',
 'eAllStructuralFeatures',
 'eAllSuperTypes',
 'eAnnotations',
 'eAttributes',
 'eGenericSuperTypes',
 'eOperations',
 'ePackage',
 'eReferences',
 'eStructuralFeatures',
 'eSuperTypes',
 'eTypeParameters',
 'eURIFragment',
 'findEOperation',
 'findEStructuralFeature',
 'getEAnnotation',
 'instanceClass',
 'instanceClassName',
 'instanceTypeName',
 'interface',
 'name',
 'notifyChanged']

In [8]:
def get_eclass(eclass_name):
    return [the_class for the_class in all_classes if the_class.name == eclass_name][0]

In [9]:
owning_membership = [the_class for the_class in all_classes if the_class.name == "OwningMembership"]

In [10]:
owning_membership[0]

<EClass name="OwningMembership">

In [11]:
owning_membership[0].eAttributes

[<EAttribute ownedMemberElementId: <pyecore.ecore.EProxy object at 0x000002907DAE5880>>,
 <EAttribute ownedMemberShortName: <pyecore.ecore.EProxy object at 0x000002907DAE5880>>,
 <EAttribute ownedMemberName: <pyecore.ecore.EProxy object at 0x000002907DAE5880>>]

In [12]:
owning_membership[0].eReferences

[<EReference ownedMemberElement: <EClass name="Element">>]

In [13]:
owning_membership[0].eAttributes[0]

<EAttribute ownedMemberElementId: <pyecore.ecore.EProxy object at 0x000002907DAE5880>>

In [14]:
dir(owning_membership[0].eAttributes[0])

['changeable',
 'defaultValueLiteral',
 'derived',
 'eAnnotations',
 'eContainingClass',
 'eGenericType',
 'eType',
 'eURIFragment',
 'getEAnnotation',
 'get_default_value',
 'iD',
 'lower',
 'lowerBound',
 'name',
 'notifyChanged',
 'notifyChanged',
 'ordered',
 'required',
 'transient',
 'unique',
 'unsettable',
 'upper',
 'upperBound',
 'volatile']

In [15]:
owning_membership[0].eAttributes[0].derived

True

In [16]:
attr_type = None
if isinstance(owning_membership[0].eAttributes[0].eType, EProxy):
    attr_type = owning_membership[0].eAttributes[0].eType.force_resolve()
else:
    attr_type = owning_membership[0].eAttributes[0].eType

In [17]:
attr_type.name

'String'

In [18]:
[super_type.name for super_type in all_classes[0].eAllSuperTypes()]

['StateUsage',
 'PerformActionUsage',
 'ActionUsage',
 'OccurrenceUsage',
 'Step',
 'Usage',
 'Feature',
 'Type',
 'Namespace',
 'Element',
 'EventOccurrenceUsage']

In [19]:
owning_membership[0].eAllAttributes()

{<EAttribute aliasIds: <pyecore.ecore.EProxy object at 0x000002907DAE5880>>,
 <EAttribute declaredName: <pyecore.ecore.EProxy object at 0x000002907DAE5880>>,
 <EAttribute declaredShortName: <pyecore.ecore.EProxy object at 0x000002907DAE5880>>,
 <EAttribute elementId: <pyecore.ecore.EProxy object at 0x000002907DAE5880>>,
 <EAttribute isImplied: <pyecore.ecore.EProxy object at 0x000002907DAE52B0>>,
 <EAttribute isImpliedIncluded: <pyecore.ecore.EProxy object at 0x000002907DAE52B0>>,
 <EAttribute isLibraryElement: <pyecore.ecore.EProxy object at 0x000002907DAE52B0>>,
 <EAttribute memberElementId: <pyecore.ecore.EProxy object at 0x000002907DAE5880>>,
 <EAttribute memberName: <pyecore.ecore.EProxy object at 0x000002907DAE5880>>,
 <EAttribute memberShortName: <pyecore.ecore.EProxy object at 0x000002907DAE5880>>,
 <EAttribute name: <pyecore.ecore.EProxy object at 0x000002907DAE5880>>,
 <EAttribute ownedMemberElementId: <pyecore.ecore.EProxy object at 0x000002907DAE5880>>,
 <EAttribute ownedMe

In [20]:
owning_membership[0].eAllReferences()

{<EReference documentation: <EClass name="Documentation">>,
 <EReference memberElement: <EClass name="Element">>,
 <EReference membershipOwningNamespace: <EClass name="Namespace">>,
 <EReference ownedAnnotation: <EClass name="Annotation">>,
 <EReference ownedElement: <EClass name="Element">>,
 <EReference ownedMemberElement: <EClass name="Element">>,
 <EReference ownedRelatedElement: <EClass name="Element">>,
 <EReference ownedRelationship: <EClass name="Relationship">>,
 <EReference owner: <EClass name="Element">>,
 <EReference owningMembership: <EClass name="OwningMembership">>,
 <EReference owningNamespace: <EClass name="Namespace">>,
 <EReference owningRelatedElement: <EClass name="Element">>,
 <EReference owningRelationship: <EClass name="Relationship">>,
 <EReference relatedElement: <EClass name="Element">>,
 <EReference source: <EClass name="Element">>,
 <EReference target: <EClass name="Element">>,
 <EReference textualRepresentation: <EClass name="TextualRepresentation">>}

In [21]:
list(owning_membership[0].eAllReferences())[10].lowerBound

0

In [22]:
def element_has_feature_by_name(element, name):
    all_refs = element.eAllReferences()
    all_atts = element.eAllAttributes()
    
    for ref in all_refs:
        if ref.name == name:
            return True
        
    for att in all_atts:
        if att.name == name:
            return True
        
    return False

In [23]:
element_has_feature_by_name(owning_membership[0], "relatedElement")

True

In [24]:
def layout_features(element):
    
    feature_line = []
    
    all_refs = element.eAllReferences()
    all_atts = element.eAllAttributes()
    
    local_refs = element.eReferences
    local_atts = element.eAttributes
    
    for ref in all_refs:
        new_line = []
        new_line.append(ref.name)
        if ref in local_refs:
            new_line.append('local')
        else:
            new_line.append('inherited')
        if ref.derived:
            new_line.append('derived')
        else:
            new_line.append('primary')
        if isinstance(ref.eType, EProxy):
            new_line.append(ref.eType.force_resolve().name)
        else:
            new_line.append(ref.eType.name)
        
        new_line.append('EReference')
        
        lb = ref.lowerBound
        ub = ref.upperBound
        
        new_line.append(lb)
        new_line.append(ub)
        
        feature_line.append(new_line)
        
    for att in all_atts:
        new_line = []
        new_line.append(att.name)
        if att in local_atts:
            new_line.append('local')
        else:
            new_line.append('inherited')
        if att.derived:
            new_line.append('derived')
        else:
            new_line.append('primary')
        if isinstance(att.eType, EProxy):
            new_line.append(att.eType.force_resolve().name)
        else:
            new_line.append(att.eType.name)
        
        new_line.append('EAttribute')
        
        lb = ref.lowerBound
        ub = ref.upperBound
        
        new_line.append(lb)
        new_line.append(ub)
        
        feature_line.append(new_line)
    
    return feature_line

In [25]:
layout_features(owning_membership[0])

[['owningRelatedElement',
  'inherited',
  'primary',
  'Element',
  'EReference',
  0,
  1],
 ['ownedAnnotation',
  'inherited',
  'derived',
  'Annotation',
  'EReference',
  0,
  -1],
 ['ownedRelatedElement',
  'inherited',
  'primary',
  'Element',
  'EReference',
  0,
  -1],
 ['ownedRelationship',
  'inherited',
  'primary',
  'Relationship',
  'EReference',
  0,
  -1],
 ['source', 'inherited', 'primary', 'Element', 'EReference', 0, -1],
 ['owningRelationship',
  'inherited',
  'primary',
  'Relationship',
  'EReference',
  0,
  1],
 ['owningNamespace', 'inherited', 'derived', 'Namespace', 'EReference', 0, 1],
 ['documentation',
  'inherited',
  'derived',
  'Documentation',
  'EReference',
  0,
  -1],
 ['textualRepresentation',
  'inherited',
  'derived',
  'TextualRepresentation',
  'EReference',
  0,
  -1],
 ['ownedMemberElement', 'local', 'derived', 'Element', 'EReference', 1, 1],
 ['target', 'inherited', 'primary', 'Element', 'EReference', 0, -1],
 ['owner', 'inherited', 'der

In [26]:
def get_primary_attributes(eclass_name):
    eclass = get_eclass(eclass_name)
    
    all_features = layout_features(eclass)
    
    primary_features = [feature for feature in all_features if feature[2] == 'primary']
    
    return primary_features

In [27]:
def get_derived_attributes(eclass_name):
    eclass = get_eclass(eclass_name)
    
    all_features = layout_features(eclass)
    
    primary_features = [feature for feature in all_features if feature[2] == 'derived']
    
    return primary_features

In [28]:
layout_features(get_eclass("OwningMembership"))

[['owningRelatedElement',
  'inherited',
  'primary',
  'Element',
  'EReference',
  0,
  1],
 ['ownedAnnotation',
  'inherited',
  'derived',
  'Annotation',
  'EReference',
  0,
  -1],
 ['ownedRelatedElement',
  'inherited',
  'primary',
  'Element',
  'EReference',
  0,
  -1],
 ['ownedRelationship',
  'inherited',
  'primary',
  'Relationship',
  'EReference',
  0,
  -1],
 ['source', 'inherited', 'primary', 'Element', 'EReference', 0, -1],
 ['owningRelationship',
  'inherited',
  'primary',
  'Relationship',
  'EReference',
  0,
  1],
 ['owningNamespace', 'inherited', 'derived', 'Namespace', 'EReference', 0, 1],
 ['documentation',
  'inherited',
  'derived',
  'Documentation',
  'EReference',
  0,
  -1],
 ['textualRepresentation',
  'inherited',
  'derived',
  'TextualRepresentation',
  'EReference',
  0,
  -1],
 ['ownedMemberElement', 'local', 'derived', 'Element', 'EReference', 1, 1],
 ['target', 'inherited', 'primary', 'Element', 'EReference', 0, -1],
 ['owner', 'inherited', 'der

In [29]:
layout_features(get_eclass("PartUsage"))

[['ownedConjugator',
  'inherited',
  'derived',
  'Conjugation',
  'EReference',
  0,
  1],
 ['nestedTransition',
  'inherited',
  'derived',
  'TransitionUsage',
  'EReference',
  0,
  -1],
 ['occurrenceDefinition',
  'inherited',
  'derived',
  'Class',
  'EReference',
  0,
  -1],
 ['multiplicity', 'inherited', 'derived', 'Multiplicity', 'EReference', 0, 1],
 ['output', 'inherited', 'derived', 'Feature', 'EReference', 0, -1],
 ['nestedConnection',
  'inherited',
  'derived',
  'ConnectorAsUsage',
  'EReference',
  0,
  -1],
 ['owningNamespace', 'inherited', 'derived', 'Namespace', 'EReference', 0, 1],
 ['nestedFlow',
  'inherited',
  'derived',
  'FlowConnectionUsage',
  'EReference',
  0,
  -1],
 ['ownedImport', 'inherited', 'derived', 'Import', 'EReference', 0, -1],
 ['ownedSubsetting',
  'inherited',
  'derived',
  'Subsetting',
  'EReference',
  0,
  -1],
 ['itemDefinition', 'inherited', 'derived', 'Structure', 'EReference', 0, -1],
 ['owningMembership',
  'inherited',
  'derive

In [30]:
def find_all_sub_types(element):
    # go through all of the meta-elements in the Ecore and find the ones that have the 
    # element given as an argument as one of their super types
    
    sub_types = []
    
    all_classes = mm_root.eClassifiers
    for clz in all_classes:
        if not isinstance(clz, EEnum):
            if element in clz.eAllSuperTypes():
                sub_types.append(clz)
            
    return sub_types

In [31]:
find_all_sub_types(get_eclass("OwningMembership"))

[<EClass name="FeatureMembership">,
 <EClass name="VariantMembership">,
 <EClass name="TransitionFeatureMembership">,
 <EClass name="StateSubactionMembership">,
 <EClass name="ViewRenderingMembership">,
 <EClass name="ObjectiveMembership">,
 <EClass name="RequirementVerificationMembership">,
 <EClass name="RequirementConstraintMembership">,
 <EClass name="SubjectMembership">,
 <EClass name="ParameterMembership">,
 <EClass name="StakeholderMembership">,
 <EClass name="FramedConcernMembership">,
 <EClass name="ActorMembership">,
 <EClass name="FeatureValue">,
 <EClass name="ReturnParameterMembership">,
 <EClass name="ResultExpressionMembership">,
 <EClass name="ElementFilterMembership">,
 <EClass name="EndFeatureMembership">]

In [32]:
get_primary_attributes("PartDefinition")

[['ownedRelationship',
  'inherited',
  'primary',
  'Relationship',
  'EReference',
  0,
  -1],
 ['owningRelationship',
  'inherited',
  'primary',
  'Relationship',
  'EReference',
  0,
  1],
 ['isAbstract', 'inherited', 'primary', 'Boolean', 'EAttribute', 0, -1],
 ['declaredName', 'inherited', 'primary', 'String', 'EAttribute', 0, -1],
 ['isVariation', 'inherited', 'primary', 'Boolean', 'EAttribute', 0, -1],
 ['declaredShortName', 'inherited', 'primary', 'String', 'EAttribute', 0, -1],
 ['isIndividual', 'inherited', 'primary', 'Boolean', 'EAttribute', 0, -1],
 ['aliasIds', 'inherited', 'primary', 'String', 'EAttribute', 0, -1],
 ['isImpliedIncluded', 'inherited', 'primary', 'Boolean', 'EAttribute', 0, -1],
 ['elementId', 'inherited', 'primary', 'String', 'EAttribute', 0, -1],
 ['isSufficient', 'inherited', 'primary', 'Boolean', 'EAttribute', 0, -1]]

In [33]:
get_derived_attributes("PartUsage")

[['ownedConjugator',
  'inherited',
  'derived',
  'Conjugation',
  'EReference',
  0,
  1],
 ['nestedTransition',
  'inherited',
  'derived',
  'TransitionUsage',
  'EReference',
  0,
  -1],
 ['occurrenceDefinition',
  'inherited',
  'derived',
  'Class',
  'EReference',
  0,
  -1],
 ['multiplicity', 'inherited', 'derived', 'Multiplicity', 'EReference', 0, 1],
 ['output', 'inherited', 'derived', 'Feature', 'EReference', 0, -1],
 ['nestedConnection',
  'inherited',
  'derived',
  'ConnectorAsUsage',
  'EReference',
  0,
  -1],
 ['owningNamespace', 'inherited', 'derived', 'Namespace', 'EReference', 0, 1],
 ['nestedFlow',
  'inherited',
  'derived',
  'FlowConnectionUsage',
  'EReference',
  0,
  -1],
 ['ownedImport', 'inherited', 'derived', 'Import', 'EReference', 0, -1],
 ['ownedSubsetting',
  'inherited',
  'derived',
  'Subsetting',
  'EReference',
  0,
  -1],
 ['itemDefinition', 'inherited', 'derived', 'Structure', 'EReference', 0, -1],
 ['owningMembership',
  'inherited',
  'derive

In [34]:
get_primary_attributes("OwningMembership")

[['owningRelatedElement',
  'inherited',
  'primary',
  'Element',
  'EReference',
  0,
  1],
 ['ownedRelatedElement',
  'inherited',
  'primary',
  'Element',
  'EReference',
  0,
  -1],
 ['ownedRelationship',
  'inherited',
  'primary',
  'Relationship',
  'EReference',
  0,
  -1],
 ['source', 'inherited', 'primary', 'Element', 'EReference', 0, -1],
 ['owningRelationship',
  'inherited',
  'primary',
  'Relationship',
  'EReference',
  0,
  1],
 ['target', 'inherited', 'primary', 'Element', 'EReference', 0, -1],
 ['memberElement', 'inherited', 'primary', 'Element', 'EReference', 1, 1],
 ['memberShortName', 'inherited', 'primary', 'String', 'EAttribute', 1, 1],
 ['declaredName', 'inherited', 'primary', 'String', 'EAttribute', 1, 1],
 ['declaredShortName', 'inherited', 'primary', 'String', 'EAttribute', 1, 1],
 ['aliasIds', 'inherited', 'primary', 'String', 'EAttribute', 1, 1],
 ['visibility', 'inherited', 'primary', 'VisibilityKind', 'EAttribute', 1, 1],
 ['isImpliedIncluded', 'inheri

In [35]:
get_primary_attributes("Membership")

[['owningRelatedElement',
  'inherited',
  'primary',
  'Element',
  'EReference',
  0,
  1],
 ['ownedRelatedElement',
  'inherited',
  'primary',
  'Element',
  'EReference',
  0,
  -1],
 ['ownedRelationship',
  'inherited',
  'primary',
  'Relationship',
  'EReference',
  0,
  -1],
 ['source', 'inherited', 'primary', 'Element', 'EReference', 0, -1],
 ['owningRelationship',
  'inherited',
  'primary',
  'Relationship',
  'EReference',
  0,
  1],
 ['target', 'inherited', 'primary', 'Element', 'EReference', 0, -1],
 ['memberElement', 'local', 'primary', 'Element', 'EReference', 1, 1],
 ['memberShortName', 'local', 'primary', 'String', 'EAttribute', 1, 1],
 ['declaredName', 'inherited', 'primary', 'String', 'EAttribute', 1, 1],
 ['declaredShortName', 'inherited', 'primary', 'String', 'EAttribute', 1, 1],
 ['aliasIds', 'inherited', 'primary', 'String', 'EAttribute', 1, 1],
 ['visibility', 'local', 'primary', 'VisibilityKind', 'EAttribute', 1, 1],
 ['isImpliedIncluded', 'inherited', 'prima

In [36]:
get_primary_attributes("Specialization")

[['owningRelatedElement',
  'inherited',
  'primary',
  'Element',
  'EReference',
  0,
  1],
 ['general', 'local', 'primary', 'Type', 'EReference', 1, 1],
 ['specific', 'local', 'primary', 'Type', 'EReference', 1, 1],
 ['ownedRelatedElement',
  'inherited',
  'primary',
  'Element',
  'EReference',
  0,
  -1],
 ['ownedRelationship',
  'inherited',
  'primary',
  'Relationship',
  'EReference',
  0,
  -1],
 ['source', 'inherited', 'primary', 'Element', 'EReference', 0, -1],
 ['owningRelationship',
  'inherited',
  'primary',
  'Relationship',
  'EReference',
  0,
  1],
 ['target', 'inherited', 'primary', 'Element', 'EReference', 0, -1],
 ['declaredName', 'inherited', 'primary', 'String', 'EAttribute', 0, 1],
 ['declaredShortName', 'inherited', 'primary', 'String', 'EAttribute', 0, 1],
 ['aliasIds', 'inherited', 'primary', 'String', 'EAttribute', 0, 1],
 ['isImpliedIncluded', 'inherited', 'primary', 'Boolean', 'EAttribute', 0, 1],
 ['isImplied', 'inherited', 'primary', 'Boolean', 'EAttr

In [37]:
get_primary_attributes("FeatureTyping")

[['owningRelatedElement',
  'inherited',
  'primary',
  'Element',
  'EReference',
  0,
  1],
 ['type', 'local', 'primary', 'Type', 'EReference', 1, 1],
 ['source', 'inherited', 'primary', 'Element', 'EReference', 0, -1],
 ['ownedRelationship',
  'inherited',
  'primary',
  'Relationship',
  'EReference',
  0,
  -1],
 ['typedFeature', 'local', 'primary', 'Feature', 'EReference', 1, 1],
 ['specific', 'inherited', 'primary', 'Type', 'EReference', 1, 1],
 ['ownedRelatedElement',
  'inherited',
  'primary',
  'Element',
  'EReference',
  0,
  -1],
 ['target', 'inherited', 'primary', 'Element', 'EReference', 0, -1],
 ['general', 'inherited', 'primary', 'Type', 'EReference', 1, 1],
 ['owningRelationship',
  'inherited',
  'primary',
  'Relationship',
  'EReference',
  0,
  1],
 ['declaredName', 'inherited', 'primary', 'String', 'EAttribute', 0, -1],
 ['declaredShortName', 'inherited', 'primary', 'String', 'EAttribute', 0, -1],
 ['aliasIds', 'inherited', 'primary', 'String', 'EAttribute', 0, 

In [38]:
attributes_dict = {k.name:get_primary_attributes(k.name) for k in all_classes if isinstance(k, EClass)}

In [39]:
file1 = open("metamodel/sysml_ecore_atts.json","w")

In [40]:
json.dump(attributes_dict, file1)

In [41]:
file1.close()

In [42]:
refs_dict = {k.name:get_derived_attributes(k.name) for k in all_classes if isinstance(k, EClass)}

In [43]:
file2 = open("metamodel/sysml_ecore_derived_refs.json","w")

In [44]:
json.dump(refs_dict, file2)

In [45]:
file2.close()